# UNDP HRM Policies RAG

## Ollama setup

In [ ]:
# Install Ollama v0.1.30
!curl https://ollama.ai/install.sh | sed 's#https://ollama.ai/download#https://github.com/jmorganca/ollama/releases/download/v0.1.30#' | sh

In [ ]:
# Setup the model as a global variable
OLLAMA_MODEL='phi:latest'

# Add the model to the environment of the operating system
import os
os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL
!echo $OLLAMA_MODEL # print the global variable to check it saved

import subprocess
import time

# Start ollama on the server ("serve")
command = "nohup ollama serve&" # "nohup" and "&" means run in the background

# Use subprocess.Popen to run the command
process = subprocess.Popen(command,
                            shell=True,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)

print("Process ID:", process.pid) # print the process ID
time.sleep(5)  # Makes Python wait for 5 seconds

!ollama -v # print the Ollama version number as a check

In [ ]:
# Query the model via the command line
# First time running it will "pull" (import) the model
!ollama run $OLLAMA_MODEL "Give me short summary of Warwick Business School."

In [ ]:
# Install prerequisites
!pip install llama-index-embeddings-huggingface
!pip install llama_index.readers.json
!pip install llama-index-llms-ollama
!pip install llama-index-vector-stores-chroma
!pip install llama-index ipywidgets
!pip install llama-index-llms-huggingface
!pip install chromadb

In [ ]:
# Import required modules from the llama_index library
from llama_index.core import VectorStoreIndex, ChatPromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import StorageContext
from llama_index.readers.json import JSONReader
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.node_parser import SentenceSplitter

# Import ChromaVectorStore and chromadb module
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# Import the Ollama class
from llama_index.llms.ollama import Ollama

In [ ]:
# Use the global variable (OLLAMA_MODEL) as our LLM
# Set a timeout of 4 minutes
llm = Ollama(model=OLLAMA_MODEL, request_timeout=240.0)

## Knowledgebase Sourcing

In [ ]:
# Load the data from the JSON file containing the policy documents
reader = JSONReader()
docs = reader.load_data("policies.json")

## Embedding Model Initialization

In [ ]:
# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Specify the LLM and embedding model in LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

## Chunking

In [ ]:
# Split the documents into chunks of 128 tokens with 20 tokens overlap
parser = SentenceSplitter(chunk_size=128, chunk_overlap=20)
docs_nodes = parser.get_nodes_from_documents(docs)

## Storage

In [ ]:
# Create client ("db") and a database ("chroma_db")
db = chromadb.PersistentClient(path="./chroma_db")
# Create a collection/table in the db
chroma_collection = db.create_collection("undp")
# Set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# Specify Chroma as our vector db
storage_context = StorageContext.from_defaults(vector_store=vector_store)

## Index Creation

In [ ]:
# Create the index
index = VectorStoreIndex(
    docs_nodes,
    storage_context = storage_context,
    embed_model = embed_model
)

# Print the metadata
print(chroma_collection)
# Print the name of the collection (table)
print(f'Collection name is: {chroma_collection.name}')

## Query Pipeline

In [ ]:
# Convert the index to a query engine with tree summarization response mode
query_engine = index.as_query_engine(response_mode="tree_summarize")

# Define the QA prompt string
qa_prompt_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "If you can't find adequate information in the context, apologize and explain that no relevant information was found."
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str),
]

# Create a chat prompt template with the defined messages
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

## Testing the System

In [ ]:
print(
    index.as_query_engine(
        text_qa_template=text_qa_template,
        llm=llm,
    ).query("Who is eligible for rental subsidy?")
)

In [ ]:
print(
    index.as_query_engine(
        text_qa_template=text_qa_template,
        llm=llm,
    ).query("Must salary advances be approved? If so, by whom?")
)

In [ ]:
print(
    index.as_query_engine(
        text_qa_template=text_qa_template,
        llm=llm,
    ).query("Can an employee absent herself to breastfeed?")
)

In [ ]:
print(
    index.as_query_engine(
        text_qa_template=text_qa_template,
        llm=llm,
    ).query("What is the policy on the use of personal drones for work-related purposes?")
)